# Computational Methods in Economics

## Problem Set 3 - Numerical Optimization 

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-03-02 15:23:35.019445


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import numpy as np
import scipy.optimize
import scipy.linalg

## Question 1 (N)

*From Judd(1998), chapter 4, question 2*. One of the classical uses of optimization is the computation of the *Pareto frontier*. Consider the endowment economy with $m$ goods and $n$ agents. Assume that agent $i$'s utility function over the $m$ goods is 

\begin{equation}
    u^{i}(x^i) = \sum^{m}_{j = 1} a^i_j (x^i_j)^{v^i_j + 1} (1 + v^i_j)^{-1} 
\end{equation}

Suppose that agent $i$'s endowment of good $j$ is $e^i_j$. Assume that $a^i_j, e^i_j > 0 > v^i_j$ (for $v^i_j$, we replace $(x^i_j)^{v^i_j + 1} (1 + v^i_j)^{-1}$ with $\ln x^i_j$). 

(a) Write a program using Scipy's BFGS implementation that will read in the $v^i_j$, $a^i_j$ and $e^i_j$ and the social weights $\lambda^i$, and output the solution to the social planner's problem. Choose $m = n = 2$ and solve the problem *analytically* for $\lambda_1 = \lambda_2 = 0.5$ and the following values for the remaining parameters: 

In [3]:
E = np.array([[6, 4], [5, 1]])
V = np.array([[-4, -2], [-3, -3]])
A = np.array([[1, 4], [1, 8]])

print("V = {}".format(V) )
print("A = {}".format(A) )
print("E = {}".format(E) )

V = [[-4 -2]
 [-3 -3]]
A = [[1 4]
 [1 8]]
E = [[6 4]
 [5 1]]


The way to read these matrices is that a good corresponds to a row and an agent to a column. For example, agent 1's endowment of good 2, $e^1_2$, would be the element in the second row and first column of matrix **E**, and hence $e^1_2 = 5$.

With these parameter values, confirm that your analytical result equals the numerical output of your program. 

(b) Test your program for higher numbers of goods and agents. You can create the parameter matrices above using Numpy's **np.random.uniform** function. Can your program handle $m = n = 5$? $m = n = 10$?      


**Hint**: A slightly tricky issue when answering this question using *unconstrained* numerical optimization methods is how to deal with the constraint that aggregate consumption of good $j$ must equal aggregate endowments, i.e.

\begin{equation}
    \sum^{n}_{i = 1} x_j^i = \sum^{n}_{i = 1} e_j^i
\end{equation}

One way to address this is to have the algorithm solve for the optimal consumption for $n - 1$ agents and evaluate the consumption and hence the utility of the last agent *as the residual*. Formally, for good $j$,

\begin{equation}
    x_j^n = \sum^{n}_{i = 1} e_j^i - \sum^{n-1}_{i = 1} x_j^i
\end{equation}

## Question 2 (N)

Consider the neoclassical growth model from the lecture. In this question, we extend it so that the production function contains *energy* $m_t$ as a third production factor in addition to capital and labor. Hence, output is given by

\begin{equation}
    y_t = f(k_t, h_{y,t}, m_t) = A k_t^\alpha m_t^\gamma h_{y,t}^{1-\alpha-\gamma}
\end{equation}

Energy is itself produced by using a part of the labor supply:

\begin{equation}
    m_t = \rho h_{m,t}
\end{equation}

which implies that one unit of labor supply creates $\rho$ units of energy.

Solve the planner problem numerically for $T = 30$. Note that lifetime utility is still given by 

\begin{equation}
    u(c_t, h_t) = \frac{c^{1-\nu}}{1-\nu} - B \frac{h_t^{1+\eta}}{1+\eta}
\end{equation}

with $h_t = h_{y,t} + h_{m,t}$. You can use the parameter values from the lecture, and $\gamma = 0.05$ and $\rho = 0.9$. 

In addition, compute the steady state using a root finding algorithm and verify that the planner's sequences for $k_t$, $h_{y,t}$ and $h_{m,t}$ converge to their steady state values.

## Question 3 (N)

In this question, we are going to apply the gradient descent minimization algorithm on a least-squares regression problem. Consider the Bundesliga data set used in the *Introduction to Python* section of this class. Let's assume we would like to regress a player's market value on his age, his number of goals and assists. Running the following cell (i) reads in the relevant columns of the data set; (ii) creates a matrix **X** with the explanatory variables; and (iii) creates an array **y** containing the dependent variables.

In [4]:
cols=(2,4,5,6)
D = np.loadtxt('BundesligaData.txt', delimiter=';',usecols=(cols), skiprows=1)
D[:10, :]

description = ['name', 'position', 'value', 'valuemax', 'age', 'goals','assists', 'yellow', 'red', 'shotspergame','passsuccess','aerialswon', 'rating', 'positioncode']
for i in cols:
    print((i,description[i]))

X = D[:,1:]
## dependent variable
y = D[:,0] 
y.shape=(D.shape[0], 1)
# Before regressing the values, we should check whether X and y have the right shape
print(X.shape)
print(y.shape)

print(X[:10, :])  

(2, 'value')
(4, 'age')
(5, 'goals')
(6, 'assists')
(291, 3)
(291, 1)
[[ 28.  30.   4.]
 [ 27.  29.   2.]
 [ 27.   5.  12.]
 [ 27.   5.   4.]
 [ 26.   4.   3.]
 [ 20.   6.  11.]
 [ 26.   2.   2.]
 [ 28.  10.   2.]
 [ 20.   2.   1.]
 [ 20.   2.   1.]]


(a) Scale the values in **X** and add a column of ones. 

(b) For comparison, use the normal equation to compute $\mathbf{b}$.

(c) Implement the gradient descent algorithm outlined above to find $\mathbf{b}$. Assume that the step size $\alpha$ is constant. You may have to play around with $\alpha$ to find a value that gives you convergence. *Hint*: Recall that in the context of gradient descent, convergence may be slow. When implementing the algorithm above with a **while** loop, you should (as we always do) include a condition that the loop stops after a certain number of iterations, **maxit**. Make sure to set **maxit** sufficiently high in order to get convergence. 